# Supplementary Notebook Comparing Haar Fluctuation Analysis Algorithms

We will be using a simple Ornstein-Uhlenbeck model as the time series of interest to compare Haar fluctuation analysis algorithms.

In [ ]:
src_path = "../Blind_Review_Code_Repository_2025/src/"

In [2]:
import os
os.chdir(src_path)

# From src
## Haar Fluctuation Analysis Functions
import haarFluctuationAnalysis
import nonEquidistantHaar
import crossHaarCorrelation

# Other Imports
import matplotlib.pyplot as plt
import numpy as np
from numba import njit

### Comparing Haar Algorithms using an OU Model

In [3]:
# Define the OU Model we will be using
@njit
def OU_model(theta, mu, sigma, steps):
    dt = 1
    tsteps = np.arange(1, steps, 1)
    tmp_res = np.empty(steps)
    tmp_res[0] = mu
    for i in tsteps:
        tmp_res[i] = tmp_res[i - 1] + theta * (mu - tmp_res[i - 1]) * dt + sigma * np.sqrt(dt) * np.random.normal()
    return tmp_res

In [ ]:
# Parameters
l = 2000
iterations = 4  # Number of parameter iterations
theta_values = np.linspace(0.01, 0.2, iterations)  # Vary theta

# Create subplots for all iterations
fig, axs = plt.subplots(iterations, 2, figsize=(10,13), constrained_layout=True)

for i, theta in enumerate(theta_values):
    # Generate OU model
    y = OU_model(theta, mu=0, sigma=2, steps = l)
    x = np.arange(1, l + 1, 1)

    # My implmentation of Hébert et al. 2021
    haar_pop, _, _ = haarFluctuationAnalysis.Haar_hebert(y, x, Scales=None, q=1, overlap=0, returnFull=False, allq=False)
    # Reyes et al. 2024
    time_scales, fluctuations = nonEquidistantHaar.Haar_RMS_fluctuations(x, y)
    # Arnischith and Rothman
    ar_fluc = haarFluctuationAnalysis.arns_haar_fluct(y, x, 0.01, 0.25, 100, plot = False)
    
    # Plot OU model and Haar analysis
    axs[i, 0].plot(x, y)
    axs[i, 0].set_title(f"OU Model (Theta={theta:.4f})")
    axs[i, 0].set_xlabel("Time")
    axs[i, 0].set_ylabel("Value")
    axs[i, 0].legend()

    axs[i, 1].plot(np.log10(haar_pop[:, 0]), np.log10(haar_pop[:, 1]), label = "Hébert et al. 2021")
    axs[i, 1].plot(time_scales, fluctuations, c="green", label = "Reyes et al. 2024")
    axs[i, 1].plot(np.log10(ar_fluc[1]), np.log10(ar_fluc[0]), c="pink", label = "Arnscheidt and Rothman 2022")
    axs[i, 1].axvline(np.log10(2 * (1 / theta)), c = "black", linewidth = 1, linestyle = "dashed")
    axs[i, 1].set_title(f"Haar Analysis (Theta={theta:.4f})")
    axs[i, 1].set_xlabel("log10(Scale)")
    axs[i, 1].set_ylabel("log10(Amplitude)")

subplot_letters = [
"(a)", "(b)","(c)", "(d)",
"(e)", "(f)","(g)", "(h)",
"(i)", "(j)"
]
# Plot Letters
for i, ax_row in enumerate(axs):
    for j, ax in enumerate(ax_row):
        idx = i * 2 + j
        ax.text(-0.15, 1.1, subplot_letters[idx], transform=ax.transAxes,
            fontsize=10, fontstyle='italic', va='top', ha='left')


# Get handles and labels from one of the analysis subplots (they all use same labels)
handles, labels = axs[0, 1].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=3, fontsize=9, frameon=False, bbox_to_anchor=(0.5, -0.02))


save_path = "../Figures/Supplementary Figures/SFig8.png"
plt.savefig(save_path, format='png', dpi=600, bbox_inches='tight',transparent=False)